In [1]:
import tensorflow as tf
from tensorflow.data import Dataset
import numpy as np
from typing import Iterator, Literal, Generator, NamedTuple, TypeAlias, Callable, TypedDict
from typing_extensions import assert_type
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import sys
from PIL import Image

C:\Users\chijioke joseph\anaconda3\envs\conda_tf\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# generate a dataset containing all the image files
raw_data: Dataset = Dataset.list_files("Dataset/*/*/*.jpg", shuffle=True)  # type: ignore
data_iterator: Iterator = raw_data.as_numpy_iterator()
IMG_SIZE: tuple[int, ...] = (224, 224)

In [3]:
NUM_AGE_CLASSES: int = len(os.listdir("Dataset")) # 0 - 7
NUM_GENDER_CLASSES: int = 2 # 0 - 1

In [4]:
# create a generator function
# use the generator function to read the data from the file
# the function must end with a yield statement, and then post yield logic

class LabelTuple(NamedTuple):
    age: tf.Tensor
    gender: tf.Tensor

class DataDict(TypedDict):
    image: tf.Tensor
    age: tf.Tensor
    gender: tf.Tensor

Encode: TypeAlias = Literal["Age", "Gender"]
Gender: TypeAlias = Literal["Male", "Female"]

# '0 - 19' -> 0, '0 - 19' -> [1, 0, 0, 0, 0, 0, 0]
# 'Female' -> 0, 'Female' -> [1, 0]

def encode(string_data: str, encode_type: Encode) -> tf.Tensor:
    """
    Generates a one-hot encoding of the string_data 'argument' depending on where the 'encode_type' is "Age" or "Gender"

    :param string_data: (str): The Age or Gender class that is passed in
    :param encode_type: (Literal["Age", "Gender"]): A Literal of values "Age" when encoding age classes or "Gender" when encoding gender classes.
    :rtype: tf.Tensor
    :return: (tf.Tensor): encoded values as a tensor.
    """
    age_ranges: list[str] = os.listdir("Dataset")
    gender_ranges: list[str] = os.listdir("Dataset/0 - 19")
    if encode_type == "Age":
        if string_data not in age_ranges: sys.exit(f"{string_data} not a valid member of {age_ranges}")
        # get matching index for the age class represented by string_data
        index: int = age_ranges.index(string_data)
        # generate an array of zeros for according to shape (NUM_AGE_CLASSES)
        encoded_age: np.ndarray = np.zeros((NUM_AGE_CLASSES,))
        # change the value at the matching index to 1
        encoded_age[index] = 1
        # return the array as a tensor
        return tf.constant(encoded_age)
    else:
        if string_data not in gender_ranges: sys.exit(f"{string_data} not a valid member of {gender_ranges}")
        # get matching index for the gender class represented by string_data
        index: int = gender_ranges.index(string_data)
        # generate an array of zeros for according to shape (NUM_GENDER_CLASSES)
        encoded_gender: np.ndarray = np.zeros((NUM_GENDER_CLASSES,))
        # change the value at the matching index to 1
        encoded_gender[index] = 1
        # return the array as a tensor
        return tf.constant(encoded_gender)


def decode(confidence: tf.Tensor, decode_type: Encode) -> str | Gender:
    """
    extracts the index of the maximum score in 'confidence', and then returns the actual predicted class as a string.

    :param confidence: (tf.Tensor): A tensor of confidence scores, whose shape depends on the 'decode_type' and the number of classes it holds.
    :param decode_type: (Literal["Age", "Gender"]): A Literal of values "Age" when encoding age classes or "Gender" when encoding gender classes.
    :rtype: str | Literal["Male", "Female"]
    :return: (str | Literal["Male", "Female"]): A string of age classes or gender classes.
    """
    age_ranges: list[str] = os.listdir("Dataset")
    get_indices: Callable = lambda x: list(range(len(x)))
    age_indices: list[int] = get_indices(age_ranges)
    gender_ranges: list[str] = os.listdir("Dataset/0 - 19")
    gender_indices: list[int] = get_indices(gender_ranges)

    # convert 'confidence' tensor to a numpy array
    confidence_array: np.ndarray = confidence.numpy()
    # get the index of the maximum confidence in the tensor
    index: int = np.argmax(confidence_array).item()
    if decode_type == "Age":
        if index not in age_indices: sys.exit(f"{index} not a valid member of {age_indices}")
        return age_ranges[index]
    else:
        if index not in gender_indices: sys.exit(f"{index} not a valid member of {gender_indices}")
        gender_return: str = gender_ranges[index]
        assert_type(gender_return, Gender)
        return gender_return

print(f"{encode('0 - 19', 'Age') = }")
print(f"{encode('30 - 39', 'Age') = }")
print(f"{encode('Male', 'Gender') = }")
print(f"{decode(tf.constant([0.25, 0.75]), 'Gender') = }")
print(f"{decode(tf.constant([0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.3, 0.1]), 'Age') = }")

encode('0 - 19', 'Age') = <tf.Tensor: shape=(8,), dtype=float64, numpy=array([1., 0., 0., 0., 0., 0., 0., 0.])>
encode('30 - 39', 'Age') = <tf.Tensor: shape=(8,), dtype=float64, numpy=array([0., 0., 1., 0., 0., 0., 0., 0.])>
encode('Male', 'Gender') = <tf.Tensor: shape=(2,), dtype=float64, numpy=array([0., 1.])>
decode(tf.constant([0.25, 0.75]), 'Gender') = 'Male'
decode(tf.constant([0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.3, 0.1]), 'Age') = '50 - 59'


In [5]:
num_images: int = len(list(raw_data))
images_to_use: int = 5000
BATCH_SIZE: int = 32

In [6]:
def read_from_iterator(iterator_object: Iterator, max_images: int | None = None) -> Generator[DataDict, None, None]:
    """Generator function that yields images, labels from the paths that it finds in an Iterator object. The data is returned as a dictionary or a special form 'DataDict'

    :param iterator_object: (Iterator): An iterator usually gotten from calling as_numpy_iterator() on a tf.data.Dataset
    :param max_images: (int, optional): The maximum number of images to read into the program. Defaults to None, in which case, it reads all the images available in the program.
    :rtype: Generator[DataDict, None, None]
    :return: (Generator[DataDict, None, None]): A generator that yields the underlying data parsed from the iterator of image paths.
    """
    count: int = 0
    if max_images is None: max_images = num_images
    for iterator_data in iterator_object:
        count += 1
        if count > max_images: break
        # each data in 'iterator_data' object is a byte
        file_path_bytes: bytes = iterator_data
        # decode bytes to string
        file_path: str = file_path_bytes.decode()
        # extract metadata from file path as a string
        metadata_list: list[str] = file_path.split(os.sep)
        # extract labels from metadata
        _, age_class, gender_value, *_ = metadata_list
        gender_class: Literal["Male", "Female"] = gender_value

        # read image and convert to tensor
        pillow_image: Image.Image = Image.open(file_path)
        image_as_array: np.ndarray = np.array(pillow_image)
        image: tf.Tensor = tf.constant(image_as_array, dtype=tf.float32)
        # resize image to the 'IMG_SIZE'
        image = tf.image.resize(image, IMG_SIZE)
        # normalize the image
        image /= 255

        image_label: LabelTuple = LabelTuple(
            age=encode(age_class, "Age"),
            gender=encode(gender_class, "Gender"),
        )
        yield DataDict(
            image=image,
            age=image_label.age,
            gender=image_label.gender,
        )

def read() -> Generator[DataDict, None, None]:
    return read_from_iterator(raw_data.as_numpy_iterator(), max_images=images_to_use)
# gen = read(data_iterator)

In [7]:
# Dataset of strings which contain files -
# Parse the Dataset of strings to get images and labels
# build the actual Dataset for training
#
ds: Dataset = Dataset.from_generator(
    read,
    output_signature={
        "image": tf.TensorSpec(shape=(*IMG_SIZE, 3)),
        "age": tf.TensorSpec(shape=(NUM_AGE_CLASSES,)),
        "gender": tf.TensorSpec(shape=(NUM_GENDER_CLASSES,)),
    },
)

In [8]:
for entry in ds:
    entry: DataDict = entry
    label_tuple: LabelTuple = LabelTuple(entry["age"], entry["gender"])
    img: tf.Tensor = entry["image"]
    img_array: np.ndarray = img.numpy()
    break

ds: Dataset = Dataset.from_generator(
    read,
    output_signature={
        "image": tf.TensorSpec(shape=(*IMG_SIZE, 3)),
        "age": tf.TensorSpec(shape=(NUM_AGE_CLASSES,)),
        "gender": tf.TensorSpec(shape=(NUM_GENDER_CLASSES,)),
    },
)

In [9]:
subset_ds, test_ds = tf.keras.utils.split_dataset(ds, left_size=0.9)
train_ds, validate_ds = tf.keras.utils.split_dataset(subset_ds, left_size=0.2)

In [10]:
pretrained_model: tf.keras.Model = MobileNetV2()
pretrained_input: tf.Tensor = pretrained_model.input
pretrained_model.trainable = False
pretrained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [11]:
new_input: tf.Tensor = tf.keras.Input((*IMG_SIZE, 3), name="input")
features_output: tf.Tensor = pretrained_model(new_input)

age_softmax_layer: layers.Dense = layers.Dense(NUM_AGE_CLASSES, activation="softmax", name="age_output")
age_output: tf.Tensor = age_softmax_layer(features_output)

gender_layer: layers.Dense = layers.Dense(NUM_GENDER_CLASSES, activation="sigmoid", name="gender_output")
gender_output: tf.Tensor = gender_layer(features_output)

new_model: tf.keras.Model = tf.keras.Model(
    inputs=new_input,
    outputs=[age_output, gender_output],
)
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 1000)        3538984     ['input[0][0]']                  
 al)                                                                                              
                                                                                                  
 age_output (Dense)             (None, 8)            8008        ['mobilenetv2_1.00_224[0][0]']   
                                                                                              

In [12]:
new_model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["categorical_accuracy"],
)

In [13]:
class LabelDict(TypedDict):
    age_output: tf.Tensor
    gender_output: tf.Tensor

def extract_data(ds_arg: Dataset) -> tuple[tf.Tensor, LabelDict]:
    images_list: list[tf.Tensor] = []
    ages_list: list[tf.Tensor] = []
    genders_list: list[tf.Tensor] = []
    for item in ds_arg.as_numpy_iterator():
        item: DataDict = item
        images_list.append(item["image"])
        ages_list.append(item["age"])
        genders_list.append(item["gender"])
    images_data: tf.Tensor = tf.constant(images_list)
    age_data: tf.Tensor = tf.constant(ages_list)
    gender_data: tf.Tensor = tf.constant(genders_list)
    return images_data, LabelDict(age_output=age_data, gender_output=gender_data)

In [14]:
# train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.cache()
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
# validate_ds = validate_ds.batch(BATCH_SIZE, drop_remainder=True)
validate_ds = validate_ds.cache()
validate_ds = validate_ds.prefetch(tf.data.AUTOTUNE)

In [15]:
train_images, train_labels = extract_data(train_ds)
validate_images, validate_labels = extract_data(validate_ds)

In [ ]:
history: tf.keras.callbacks.History = new_model.fit(
    train_images,
    train_labels,
    batch_size = BATCH_SIZE,
    validation_data=(validate_images, validate_labels),
    epochs=50,
    validation_batch_size=BATCH_SIZE,
)

Epoch 1/50
29/29 [==============================] - 138s 4s/step - loss: 2.7476 - age_output_loss: 2.0593 - gender_output_loss: 0.6883 - age_output_categorical_accuracy: 0.3044 - gender_output_categorical_accuracy: 0.5811 - val_loss: 2.7257 - val_age_output_loss: 2.0421 - val_gender_output_loss: 0.6836 - val_age_output_categorical_accuracy: 0.3292 - val_gender_output_categorical_accuracy: 0.5667
Epoch 2/50
29/29 [==============================] - 104s 4s/step - loss: 2.6981 - age_output_loss: 2.0222 - gender_output_loss: 0.6759 - age_output_categorical_accuracy: 0.3822 - gender_output_categorical_accuracy: 0.5911 - val_loss: 2.6826 - val_age_output_loss: 2.0082 - val_gender_output_loss: 0.6743 - val_age_output_categorical_accuracy: 0.3686 - val_gender_output_categorical_accuracy: 0.5625
Epoch 3/50
29/29 [==============================] - 108s 4s/step - loss: 2.6532 - age_output_loss: 1.9880 - gender_output_loss: 0.6652 - age_output_categorical_accuracy: 0.4044 - gender_output_categoric

In [164]:
history.history

{'loss': [2.7727205753326416,
  2.7661654949188232,
  2.7596895694732666,
  2.753736972808838,
  2.747657537460327],
 'age_output_loss': [2.0769293308258057,
  2.0724072456359863,
  2.067990303039551,
  2.0639498233795166,
  2.0595107078552246],
 'gender_output_loss': [0.6957910656929016,
  0.6937580704689026,
  0.6916990280151367,
  0.6897873878479004,
  0.6881468296051025],
 'age_output_categorical_accuracy': [0.1666666716337204,
  0.31111112236976624,
  0.41111111640930176,
  0.42222222685813904,
  0.4000000059604645],
 'gender_output_categorical_accuracy': [0.30000001192092896,
  0.4888888895511627,
  0.5444444417953491,
  0.6111111044883728,
  0.6222222447395325],
 'val_loss': [2.771172523498535,
  2.765810489654541,
  2.7605319023132324,
  2.755241632461548,
  2.7500085830688477],
 'val_age_output_loss': [2.0762033462524414,
  2.072394847869873,
  2.068638801574707,
  2.0648481845855713,
  2.061093807220459],
 'val_gender_output_loss': [0.6949690580368042,
  0.6934154629707336,
 

In [ ]:
class HistoryDict(TypedDict):
    loss: list[float]
    age_output_loss: list[float]
    gender_output_loss: list[float]
    age_output_categorical_accuracy: list[float]
    gender_output_categorical_accuracy: list[float]
    val_loss: list[float]
    val_age_output_loss: list[float]
    val_gender_output_loss: list[float]
    val_age_output_categorical_accuracy: list[float]
    val_gender_output_categorical_accuracy: list[float]


def plot_set(metrics: list[float], val_metrics: list[float], name: str) -> None:
    plt.plot(metrics, range(len(metrics)))
    plt.plot(val_metrics, range(len(val_metrics)))
    plt.xlabel("Epoch")
    plt.ylabel(name)
    plt.legend(["Loss", "Validation Loss"])
    plt.title("Loss and Validation Loss")
    plt.grid(visible=True, which="both")
    plt.show()

def plot_history(train_history: HistoryDict) -> None:
    losses: list[float] = train_history["loss"]
    val_losses: list[float] = train_history["val_loss"]
    plot_set(losses, val_losses, "Loss")

    age_losses: list[float] = train_history["age_output_loss"]
    val_age_output_losses: list[float] = train_history["val_age_output_loss"]
    plot_set(age_losses, val_age_output_losses, "Age Output Loss")

    gender_losses: list[float] = train_history["gender_output_loss"]
    val_gender_output_losses: list[float] = train_history["val_gender_output_loss"]
    plot_set(gender_losses, val_gender_output_losses, "Gender Output Loss")

    age_accuracy: list[float] = train_history["age_output_categorical_accuracy"]
    val_age_accuracy: list[float] = train_history["val_age_output_categorical_accuracy"]
    plot_set(age_accuracy, val_age_accuracy, "Age Output Accuracy")

    gender_accuracy: list[float] = train_history["gender_output_categorical_accuracy"]
    val_gender_accuracy: list[float] = train_history["val_gender_output_categorical_accuracy"]
    plot_set(gender_accuracy, val_gender_accuracy, "Gender Output Accuracy")


plot_history(history.history)